In [6]:
import pandas as pd
import numpy as np
import regex as re
from pandarallel import pandarallel
# import os
# from pathlib import Path
# from natsort import natsorted, ns
import glob
import pydash as dash
from polyfuzz import PolyFuzz
import rapidfuzz as fuzz
from rapidfuzz.process import extractOne
from rapidfuzz.distance import Levenshtein
from rapidfuzz.fuzz import ratio
from rapidfuzz import utils as fuzz_utils
# import itertools as it
# import json
# import spacy
# from polyfuzz import PolyFuzz
# from polyfuzz.models import EditDistance, TFIDF, RapidFuzz #Embeddings, SpacyEmbeddings
# from flair.embeddings import TransformerWordEmbeddings
# import tfidf_matcher as matcher
from ftfy import fix_text
# from textblob import TextBlob, Word, Blobber
# from collections import namedtuple, defaultdict, deque
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


pandarallel.initialize(progress_bar=False, verbose=1)
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.float_format', lambda x: '%.0f' % x)


In [ ]:
# df_in = pd.read_excel('infiles/shopify_orders.xlsx', dtype=str)
df_in = pd.read_excel('infiles/odoo_orders_all.xlsx', dtype=str)

# df_in

frame = df_in[['orderId', 'stockCode', 'items', 'itemQty']].copy()
frame['itemQty'] = frame['itemQty'].astype(np.int32)
# frame.head(15)

# frame.info()
# frame['items'] = frame['items'].str.replace(rf"^{frame['orderId'].astype(str)} - ", '', regex=True)
frame['items'] = frame[['orderId', 'items']].astype(str).parallel_apply(lambda x: re.sub(rf"{x['orderId']} - ", '', x['items']), axis=1)

frame['items'] = frame['items'].str.strip()

# frame


frame.loc[frame['orderId'].str.startswith('#'), 'orderId'] = frame['orderId'].str.replace('#', 'SH_')
frame.loc[frame['orderId'].str.startswith('WAL'), 'orderId'] = frame['orderId'].str.replace(r'WAL_ |WAL-_ ', 'WAL_')
# frame.loc[frame['orderId'].str.startswith('WAL'), 'orderId'] = frame['orderId'].str.replace('WAL_ ', 'WAL_')

frame = frame.loc[frame['orderId'].str.startswith('TAM') == False].reset_index(drop=True).copy()

# frame

# frame.loc[frame['orderId'].apply(lambda x: re.search(r'^SH|^WAL', x) == True)]
# frame.loc[~frame['orderId'].str.contains(r'SH_|WAL_', regex=True, case=False)]
frame.loc[frame['orderId'].str.contains(r'^S\d', regex=True), 'orderId'] = frame['orderId'].str.replace(r'^S(\d+)', r'EB\1', regex=True)

# frame.loc[frame['orderId'].str.contains('EB')]

with pd.ExcelWriter('infiles/odoo_orders_all.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
    frame.to_excel(writer, index = False, sheet_name='update')

df = df_in.copy()
db = df_in.copy()

df.item_qty = df.item_qty.astype(int)
db.item_qty = db.item_qty.astype(int)

df['item'] = df['item'].str.replace(r'^New\s', '', regex=True).str.strip().str.replace(r'Everest Brand\s|Everest\s','', case=False, regex=True).str.strip()
db['item'] = db['item'].str.replace(r'^New\s', '', regex=True).str.strip().str.replace(r'Everest Brand\s|Everest\s','', case=False, regex=True).str.strip()

basket = (df.groupby(['order_id', 'item'])['item_qty'].sum().unstack().reset_index().fillna(0).set_index('order_id'))
bkt = (db.groupby(['order_id', 'barcode'])['item_qty'].sum().unstack().reset_index().fillna(0).set_index('order_id'))

def encodeUnits(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

bf_plus = basket.parallel_applymap(encodeUnits)
bs_filter_plus = bf_plus[(bf_plus > 0).sum(axis=1) >= 2]

bkt_plus = bkt.parallel_applymap(encodeUnits)
bkt_filter_plus = bkt_plus[(bkt_plus > 0).sum(axis=1) >= 2]

bkt_filter_plus.head()

freq_itms = apriori(bs_filter_plus, min_support=.02, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
freq_bkt = apriori(bkt_filter_plus, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
freq_bkt.shape
freq_bkt['length'] = freq_itms['itemsets'].apply(lambda x: len(x))
freq_bkt

af = association_rules(freq_bkt, metric='lift', min_threshold=1).sort_values(by='lift', ascending=False).reset_index(drop=True)
af



In [ ]:
# # products = df.loc[df[['barcode', 'item']].duplicated() == False]
# products.set_index('item', inplace=True)
# products.head()
# df.head()
